# Building the dataset for PerfectCity.io

PerfectCity.io requires many features for cities. Often we would need to look at multiple datasets for each city and extract relevant information and each dataset requires careful treatment to get the right data. This notebook will document the process that went from basically having no data to a complete PerfectCity.io dataset

In [2]:
import pandas as pd
import json

In [39]:
# define cities and the parameters we are going to use
cities = ['VANCOUVER', 'MONTREAL', 'TORONTO', 'OTTAWA', 'HAMILTON', 'WINNIPEG', 'EDMONTON', 'CALGARY' , 'SASKATOON']
parameters = ['PARKS', 'TRANSIT_SCORE', 'OUTDOOR', 'POPULATION', 'BUSINESSES' ,'CRIME', 'UNIVERSITY']

print 'cities x parameters '
print len(cities),'x',len(parameters),'=',len(cities)*len(parameters)

cities x parameters 
9 x 7 = 63


In [4]:
# build an empty object to hold the information
data = { city: { parameter : 0 for parameter in parameters } for city in cities }

#### Dealing with Accents
Many Canadian cities have accents in their names, we use the REGEX in the following way to match the first 4 charaters of the city name which seemed to produce good enough results for our purposes    
```
search_city = city[:4] # select the first four letters of the city name
search_term = r'\b('+search_city+')\w?' # incorporate it into the regex
```

---
# Transit Score Ranking
Obtained from [WalkScore.com 2014 Transit Score Ranking](http://blog.walkscore.com/2014/03/best-canadian-cities-for-public-transit/#.VYWKBxNJaV4)

In [5]:
# RANKING FROM http://blog.walkscore.com/2014/03/best-canadian-cities-for-public-transit/#.VYWKBxNJaV4
TRANSIT_SCORE = {'TORONTO': 78, 'MONTREAL': 77, 'VANCOUVER': 74, 'WINNIPEG': 51, 'OTTAWA': 49, 'EDMONTON': 44, 'CALGARY':43, 'HAMILTON': 42 }

for city, score in TRANSIT_SCORE.items():
    data[city]['TRANSIT_SCORE'] = score/100.0

----
# Population Information
[Stats Canada census from 2011](http://www12.statcan.gc.ca/census-recensement/2011/dp-pd/hlt-fst/pd-pl/Table-Tableau.cfm?LANG=Eng&T=205&S=3&RPP=50)

In [6]:
population_df = pd.read_csv('./datasets/canadian_population_census_2011.CSV')
print(population_df.columns[[1,4]]) # we need to keep columns 1 and 4 and delete everything else
population_df = population_df[[1,4]]
population_df['Geographic name'] = population_df['Geographic name'].str.upper()
population_df.head() #all good

Index([u'Geographic name', u'Population, 2011'], dtype='object')


,Geographic name,"Population, 2011"
0,TORONTO (ONT.),5583064
1,MONTR�AL (QUE.),3824221
2,VANCOUVER (B.C.),2313328
3,OTTAWA - GATINEAU (ONT./QUE.),1236324
4,CALGARY (ALTA.),1214839


In [7]:
for city in cities:
    search_term = city[:4]
    data[city]['POPULATION'] = population_df[ population_df['Geographic name'].str.contains(r'\b('+search_term+')\w?', na=False) ]['Population, 2011'].tolist()[0]

/Library/Python/2.7/site-packages/pandas/core/strings.py:184: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


-----
# Age Groups
Curated from *[Statistics Canada. Table  051-0056 -  Estimates of population by census metropolitan area, sex and age group for July 1, based on the Standard Geographical Classification (SGC) 2011, annual (persons),  CANSIM (database). (accessed: 2015-06-20)](http://www5.statcan.gc.ca/cansim/a47)*

In [8]:
age_df = pd.read_csv('./datasets/age_groups.csv')
age_df['Geography'] = age_df['Geography'].str.upper()

age_df['Age'] = age_df['Age'].str[:-6] # get rid of the years prefix.
age_df['Age'] = age_df['Age'].str.replace(' to ', '-') # remove the space around 'to' and add the '-'
age_df['Age'] = age_df['Age'].str.replace(' years an', '+') # remove the 'years and over' part of 90

age_df.head()

,Geography,Age,2014
0,"MONTR�AL, QUEBEC [24462]",0-4,226900
1,"MONTR�AL, QUEBEC [24462]",5-9,217483
2,"MONTR�AL, QUEBEC [24462]",10-14,199491
3,"MONTR�AL, QUEBEC [24462]",15-19,227351
4,"MONTR�AL, QUEBEC [24462]",20-24,287307


In [9]:
# prepare our breakdowns

age_breakdowns = age_df[ age_df['Geography'].str.contains(r'\b('+search_term+')\w?', na=False) ]['Age'].tolist()

for key, age in enumerate(age_breakdowns):
    age_breakdowns[key] = 'AGE_CAT_'+age
    
print age_breakdowns

['AGE_CAT_0-4', 'AGE_CAT_5-9', 'AGE_CAT_10-14', 'AGE_CAT_15-19', 'AGE_CAT_20-24', 'AGE_CAT_25-29', 'AGE_CAT_30-34', 'AGE_CAT_35-39', 'AGE_CAT_40-44', 'AGE_CAT_45-49', 'AGE_CAT_50-54', 'AGE_CAT_55-59', 'AGE_CAT_60-64', 'AGE_CAT_65-69', 'AGE_CAT_70-74', 'AGE_CAT_75-79', 'AGE_CAT_80-84', 'AGE_CAT_85-89', 'AGE_CAT_90+']


In [10]:
for city in cities:
    search_term = city[:4]
    ages = age_df[ age_df['Geography'].str.contains(r'\b('+search_term+')\w?', na=False) ]['2014'].tolist()    
    total = sum(ages) # get totals so we can normalize later
    for key, age in enumerate(ages):
        ages[key] = age / float(total)

    associations =  dict( zip( age_breakdowns, ages ) )
    for category, value in associations.items():
        data[city][category] = value
        
#     data[city]['AGE_'] = population_df[ population_df['Geography'].str.contains(r'\b('+search_term+')\w?', na=False) ]['Population, 2011'].tolist()[0]

----
# Parks and Green Spaces
Curate from *[Statistics Canada. Table  153-0148 -  Households and the environment survey, parks and green spaces, Canada, provinces and census metropolitan areas (CMA), every 2 years (percent),  CANSIM (database). (accessed: 2015-06-20)](http://www5.statcan.gc.ca/cansim/a26)*

- "Close to home" is defined as being a 10 minute journey from home

In [11]:
parks_df = pd.read_csv('./datasets/canadian_parks2013.csv')
parks_df['CITY'] = parks_df['CITY'].str.upper()
parks_df.head() #all good

,CITY,PARKS
0,"MONTREAL, QUEBEC [24462]",91
1,"OTTAWA-GATINEAU, ONTARIO/QUEBEC [24505 35505]",91
2,"TORONTO, ONTARIO [35535]",85
3,"HAMILTON, ONTARIO [35537]",86
4,"WINNIPEG, MANITOBA [46602]",88


In [12]:
for city in cities:
    search_term = city[:4]
    data[city]['PARKS'] = parks_df[ parks_df['CITY'].str.contains(r'\b('+search_term+')\w?', na=False) ]['PARKS'].tolist()[0]/100.0

# Outdoor Activities
Curated from [Statistics Canada. Table  153-0153 -  Households and the environment survey, participation in outdoor activities, Canada, provinces and census metropolitan areas (CMA), every 2 years (percent),  CANSIM (database). (accessed: 2015-06-20)](http://www5.statcan.gc.ca/cansim/a47)

In [13]:
outdoor_df = pd.read_csv('./datasets/outdoor_activities2013.csv')
outdoor_df['CITY'] = outdoor_df['CITY'].str.upper()
outdoor_df.head() #all good

,CITY,OUTDOOR
0,"MONTR�AL, QUEBEC [24462]",66
1,"OTTAWA-GATINEAU, ONTARIO/QUEBEC [24505 35505]",79
2,"TORONTO, ONTARIO [35535]",69
3,"HAMILTON, ONTARIO [35537]",73
4,"WINNIPEG, MANITOBA [46602]",73


In [14]:
for city in cities:
    search_term = city[:4]
    data[city]['OUTDOOR'] = outdoor_df[ outdoor_df['CITY'].str.contains(r'\b('+search_term+')\w?', na=False) ]['OUTDOOR'].tolist()[0]/100.0

-----
# Crime severity Index

We obtain the crime severity statistic from [Statistics Canada. Table  252-0052 -  Crime severity index and weighted clearance rates, annual (index unless otherwise noted),  CANSIM (database). (accessed: 2015-06-20)](http://www5.statcan.gc.ca/cansim/a26)

In [15]:
crime_df = pd.read_csv('./datasets/crime_severity2013.csv')
crime_df['CITY'] = crime_df['CITY'].str.upper()
crime_df.head()

,CITY,CRIME_SEVERITY_INDEX
0,"MONTR�AL, QUEBEC (28,34)",65.93
1,"OTTAWA-GATINEAU, ONTARIO/QUEBEC (6)",53.28
2,"TORONTO, ONTARIO (25)",47.14
3,"HAMILTON, ONTARIO (25)",55.11
4,"WINNIPEG, MANITOBA (9,10,33)",83.17


In [16]:
for city in cities:
    search_term = city[:4]
    data[city]['CRIME'] = crime_df[ crime_df['CITY'].str.contains(r'\b('+search_term+')\w?', na=False) ]['CRIME_SEVERITY_INDEX'].tolist()[0]/100.0

# Unemployment
Curated from [Statistics Canada. Table  109-5334 -  Unemployment rate, Canada, provinces, health regions (2014 boundaries) and peer groups, annual (percent),  CANSIM (database). (accessed: 2015-06-21)](http://www5.statcan.gc.ca/cansim/a26?lang=eng&retrLang=eng&id=1095334&pattern=unemployment&tabMode=dataTable&srchLan=-1&p1=1&p2=-1)

In [34]:
unemp_df = pd.read_csv('./datasets/unemployment2014.csv')
unemp_df['CITY'] = unemp_df['CITY'].str.upper()
unemp_df.head()

,CITY,CATEGORY,RATE
0,"R�GION DE MONTR�AL, QUEBEC [2406-G]","Unemployment rate, 15 years and over",9.8
1,"R�GION DE MONTR�AL, QUEBEC [2406-G]","Unemployment rate, 15 to 24 years",13.8
2,"CITY OF HAMILTON HEALTH UNIT, ONTARIO [3537-A]","Unemployment rate, 15 years and over",6.2
3,"CITY OF HAMILTON HEALTH UNIT, ONTARIO [3537-A]","Unemployment rate, 15 to 24 years",13.7
4,"CITY OF OTTAWA HEALTH UNIT, ONTARIO [3551-B]","Unemployment rate, 15 years and over",6.7


In [37]:
for city in cities:
    search_term = city[:4]
    
    #search results
    search_result = unemp_df[ unemp_df['CITY'].str.contains(r'\b('+search_term+')\w?', na=False) ]
    
    # split into two categories, 15+ and 15-24
    data[city]['UNEMPL_15-24'] = search_result[search_result['CATEGORY'].str.contains('15 to 24')]['RATE'].tolist()[0]/100.0
    data[city]['UNEMPL_15+'] = search_result[search_result['CATEGORY'].str.contains('15 years and over')]['RATE'].tolist()[0]/100.0

# Universities
Counted from [4ICU](http://www.4icu.org/ca/universities-canada.htm). Each data point is divide by the `max()` to allow us to obtain a score from 0 to 1. This enables us to compare the number of universities amongst each other.

In [17]:
number_of_universities = { 'VANCOUVER': 2, 'MONTREAL': 7, 'TORONTO': 4, 'OTTAWA': 3, 'HAMILTON': 1, 'WINNIPEG': 3, 'EDMONTON': 5, 'CALGARY': 5, 'SASKATOON': 2 }

In [18]:
for city, n in number_of_universities.items():
    data[city]['UNIVERSITY'] = n/7.0

# Final Curated Data set

In [55]:
s = json.dumps(data)
df = pd.read_json(s, orient='index')
df


,AGE_CAT_0-4,AGE_CAT_10-14,AGE_CAT_15-19,AGE_CAT_20-24,AGE_CAT_25-29,AGE_CAT_30-34,AGE_CAT_35-39,AGE_CAT_40-44,AGE_CAT_45-49,AGE_CAT_5-9,...,BUSINESSES,CRIME,OUTDOOR,PARKS,POPULATION,TRANSIT_SCORE,UNEMPLOYMENT,UNEMPL_15+,UNEMPL_15-24,UNIVERSITY
CALGARY,0.063840,0.053175,0.056785,0.068301,0.085462,0.093434,0.082916,0.077784,0.071268,0.059873,...,0,0.6040,0.69,0.88,1214839,0.43,0,0.048,0.098,0.714286
EDMONTON,0.061928,0.051831,0.057203,0.075901,0.090961,0.089605,0.075942,0.068441,0.066505,0.057247,...,0,0.8449,0.80,0.89,1159869,0.44,0,0.051,0.100,0.714286
HAMILTON,0.050418,0.054018,0.062290,0.072376,0.066994,0.063852,0.062413,0.065816,0.071380,0.053104,...,0,0.5511,0.73,0.86,721053,0.42,0,0.062,0.137,0.142857
MONTREAL,0.056343,0.049537,0.056455,0.071343,0.069167,0.075015,0.074128,0.069141,0.071504,0.054005,...,0,0.6593,0.66,0.91,3824221,0.77,0,0.098,0.138,1.000000
OTTAWA,0.054458,0.052970,0.061301,0.076179,0.073756,0.070534,0.068428,0.069660,0.072807,0.055031,...,0,0.5328,0.79,0.91,1236324,0.49,0,0.067,0.128,0.428571
SASKATOON,0.062471,0.054315,0.058822,0.085765,0.102078,0.085642,0.070667,0.063835,0.061224,0.058353,...,0,0.9909,0.77,0.89,260600,0.00,0,0.040,0.068,0.285714
TORONTO,0.053736,0.054635,0.063571,0.072731,0.073705,0.074372,0.071559,0.073017,0.075850,0.055133,...,0,0.4714,0.69,0.85,5583064,0.78,0,0.095,0.216,0.571429
VANCOUVER,0.046733,0.048941,0.061900,0.074837,0.074465,0.076066,0.070180,0.073289,0.075123,0.048129,...,0,0.9026,0.75,0.87,2313328,0.74,0,0.047,0.091,0.285714
WINNIPEG,0.054495,0.056325,0.063847,0.074039,0.076011,0.073354,0.068369,0.066727,0.068052,0.054693,...,0,0.8317,0.73,0.88,730018,0.51,0,0.060,0.117,0.428571


It's useful to know the minimum and maximum values for each column, this code seems to extract that

In [54]:
minimums = {}
categories = map( str , df.columns.values.tolist())
for category in categories:
    minimums[category] = df[category].min()
minimums

{'AGE_CAT_0-4': 0.046732588778074007,
 'AGE_CAT_10-14': 0.048940832428918002,
 'AGE_CAT_15-19': 0.056454971181645004,
 'AGE_CAT_20-24': 0.068300679381341003,
 'AGE_CAT_25-29': 0.066994412122922009,
 'AGE_CAT_30-34': 0.063851557967037009,
 'AGE_CAT_35-39': 0.062412771095673003,
 'AGE_CAT_40-44': 0.063835095165549999,
 'AGE_CAT_45-49': 0.061223946725919007,
 'AGE_CAT_5-9': 0.048128781693154002,
 'AGE_CAT_50-54': 0.068322278917221008,
 'AGE_CAT_55-59': 0.062584404957523002,
 'AGE_CAT_60-64': 0.048416139376607006,
 'AGE_CAT_65-69': 0.035442706833835005,
 'AGE_CAT_70-74': 0.023005272545159001,
 'AGE_CAT_75-79': 0.016957164924672002,
 'AGE_CAT_80-84': 0.013060869923744001,
 'AGE_CAT_85-89': 0.0083093946845180008,
 'AGE_CAT_90+': 0.0045254176201250003,
 'BUSINESSES': 0,
 'CRIME': 0.47139999999999904,
 'OUTDOOR': 0.66000000000000003,
 'PARKS': 0.84999999999999909,
 'POPULATION': 260600,
 'TRANSIT_SCORE': 0.0,
 'UNEMPLOYMENT': 0,
 'UNEMPL_15+': 0.040000000000000001,
 'UNEMPL_15-24': 0.068000000

In [56]:
maximums = {}
categories = map( str , df.columns.values.tolist())
for category in categories:
    maximums[category] = df[category].max()
maximums

{'AGE_CAT_0-4': 0.06383995120567601,
 'AGE_CAT_10-14': 0.056324747010119004,
 'AGE_CAT_15-19': 0.063846723908820999,
 'AGE_CAT_20-24': 0.085765415754705004,
 'AGE_CAT_25-29': 0.102077609318972,
 'AGE_CAT_30-34': 0.093434305736531012,
 'AGE_CAT_35-39': 0.082915517718154003,
 'AGE_CAT_40-44': 0.077784436288361,
 'AGE_CAT_45-49': 0.075849559854445012,
 'AGE_CAT_5-9': 0.059873279776160006,
 'AGE_CAT_50-54': 0.080053287935410999,
 'AGE_CAT_55-59': 0.070929187118087003,
 'AGE_CAT_60-64': 0.058866115719759006,
 'AGE_CAT_65-69': 0.051349401746930007,
 'AGE_CAT_70-74': 0.037978223483719001,
 'AGE_CAT_75-79': 0.029815950278871003,
 'AGE_CAT_80-84': 0.024009837590887002,
 'AGE_CAT_85-89': 0.016425170014688,
 'AGE_CAT_90+': 0.0090221476474980009,
 'BUSINESSES': 0,
 'CRIME': 0.9909,
 'OUTDOOR': 0.80000000000000004,
 'PARKS': 0.91000000000000003,
 'POPULATION': 5583064,
 'TRANSIT_SCORE': 0.78000000000000003,
 'UNEMPLOYMENT': 0,
 'UNEMPL_15+': 0.098000000000000004,
 'UNEMPL_15-24': 0.2160000000000000